In [16]:
from model.model import create_model_fn
from data.triplet_generator import TripletGenerator
import tensorflow as tf
import numpy as np

In [2]:
model_params = tf.contrib.training.HParams(
    fit=True,
    file_path='/Users/deniz/Documents/whales/train.csv',
    image_train_path='/Users/deniz/Documents/whales/train',
    image_test_path='/Users/deniz/Documents/whales/test',
    model_architecture='convnet',
    tmp_data_path="/tmp/whales/",
    batch_size=None,
    nb_epochs=None,
    lr_rate=0.02,
    lr_policy=None,
    pretrained=True,
    nb_layers_to_freeze=0,
    loss=None,
    embedding_hidden_dim=64,
    triplet_margin=None,
    nb_classes=10,
    dropout=0.3)
model_params.add_hparam('image_dim', (224, 224, 3))
model_params

HParams([('batch_size', None), ('dropout', 0.3), ('embedding_hidden_dim', 64), ('file_path', '/Users/deniz/Documents/whales/train.csv'), ('fit', True), ('image_dim', (224, 224, 3)), ('image_test_path', '/Users/deniz/Documents/whales/test'), ('image_train_path', '/Users/deniz/Documents/whales/train'), ('loss', None), ('lr_policy', None), ('lr_rate', 0.02), ('model_architecture', 'convnet'), ('nb_classes', 10), ('nb_epochs', None), ('nb_layers_to_freeze', 0), ('pretrained', True), ('tmp_data_path', '/tmp/whales/'), ('triplet_margin', None)])

In [4]:
model = create_model_fn(model_params)

INFO:model.model:pretrained..


Tensor("input_2:0", shape=(?, 224, 224, 3), dtype=float32)
Tensor("conv0_relu_1/Relu6:0", shape=(?, 111, 111, 64), dtype=float32)
Tensor("conv1_relu_1/Relu6:0", shape=(?, 55, 55, 64), dtype=float32)


INFO:model.model:append the top to the structure..


Tensor("conv2_relu_1/Relu6:0", shape=(?, 27, 27, 64), dtype=float32)
Tensor("conv3_relu_1/Relu6:0", shape=(?, 13, 13, 64), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 111, 111, 64)      1728      
_________________________________________________________________
conv0_bn (BatchNormalization (None, 111, 111, 64)      256       
_________________________________________________________________
conv0_relu (ReLU)            (None, 111, 111, 64)      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 55, 55, 64)        36864     
_________________________________________________________________
conv1_bn (BatchNormalization (None, 55, 55, 64)        256       
____

INFO:tensorflow:None


In [5]:
data_params = {
    'featurewise_center': False,
    'featurewise_std_normalization': False,
    'rotation_range': 20,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1,
    'zoom_range': 0.2,
    'shear_range': 0.4,
   }

In [6]:
train_generator = TripletGenerator(file_path=model_params.file_path,
                                   image_path=model_params.image_train_path,
                                   image_test_path=model_params.image_test_path,
                                   nb_classes_batch=10,
                                   nb_images_per_class_batch=5,
                                   validation_split=0.2,
                                   subset='training',
                                   **data_params)
eval_generator = TripletGenerator(file_path=model_params.file_path,
                                  image_path=model_params.image_train_path,
                                  image_test_path=model_params.image_test_path,
                                  nb_classes_batch=10,
                                  nb_images_per_class_batch=5,
                                  validation_split=0.2,
                                  subset='eval',
                                  **data_params)

INFO:data.triplet_generator:exclude_class: ['new_whale']
INFO:data.triplet_generator:class_weight_type: None
INFO:data.triplet_generator:9664 instances excluded
INFO:data.triplet_generator:data has shape: (12415, 2)
INFO:data.triplet_generator:{'featurewise_center': False, 'featurewise_std_normalization': False, 'rotation_range': 20, 'width_shift_range': 0.1, 'height_shift_range': 0.1, 'zoom_range': 0.2, 'shear_range': 0.4, 'preprocessing_function': <function _preprocess_input at 0x129fcb488>}
INFO:data.triplet_generator:exclude_class: ['new_whale']
INFO:data.triplet_generator:class_weight_type: None
INFO:data.triplet_generator:9664 instances excluded
INFO:data.triplet_generator:data has shape: (3282, 2)
INFO:data.triplet_generator:{'featurewise_center': False, 'featurewise_std_normalization': False, 'rotation_range': 20, 'width_shift_range': 0.1, 'height_shift_range': 0.1, 'zoom_range': 0.2, 'shear_range': 0.4, 'preprocessing_function': <function _preprocess_input at 0x129fcb488>}


In [7]:
len(train_generator), len(eval_generator)

(400, 100)

In [ ]:
nex

In [17]:
def train(train_generator, eval_generator,outer_step_size = 0.02, steps=5):
    def fn(_gen):
        for a, b in _gen:
            yield a, b
    nb_steps_per_epoch = len(train_generator)
    nb_steps_per_epoch_val = len(eval_generator)
    generator = fn(train_generator)
    eval_generator = fn(eval_generator)
    NB_EPOCHS = 1
    # reptile training loop
    for i in range(NB_EPOCHS):
        for s in range(nb_steps_per_epoch):
            
            weights_before = model.weights
            # sample a new task. need to take k steps.
            x, y = next(generator)
            # randomize bc generator does not
            ix = list(range(x.shape[0]))
            np.random.shuffle(ix)
            x, y = x[ix], y[ix]

            batch_size = x.shape[0] // steps
            for s in range(steps):   
                _x, _y = x[s * batch_size: (s+1) * batch_size,:,:,:], y[s * batch_size: (s+1) * batch_size,:]
                model.train_on_batch(_x, _y)
            weights_after = model.weights
            # meta update after task.
            for w_ix in range(len(weights_before)):
                model.weights[i] = (weights_before[i] +
                                    (weights_after[i] - weights_before[i]) * outer_step_size)
            
        # eval
        print('eval')
        for s in range(nb_steps_per_epoch_val):
            outs = model.test_on_batch(x, y)
        return model

In [ ]:
model = train(train_generator, eval_generator)